In [1]:
import pandas as pd
import os
import re

In [2]:
#Loading in the text 
alltext=''
file_names=['/es/test_v3.txt','/es/dev_100_v3.txt','/es/dev_500_v3.txt']
for j in file_names:
    text=pd.read_csv(os.getcwd()[:-8]+'langs'+j,delimiter='\t',header=None,quoting=3)
    text.columns=['id', 'text']
    for i in text.iterrows():
        try:
            if int(i[1][0]):
                alltext=alltext+' '+i[1][1]
        except:
               pass
alltext_lower=''
for i in alltext:
    if i.isupper():
        alltext_lower=alltext_lower+i.lower()
    else:
        alltext_lower=alltext_lower+i

In [3]:
tokenized=[]
tokenized_strings=[]
#Loading in the tokenized data
with open(os.getcwd()[:-8]+'multilexnorm/data/es/train.norm') as text:
    temp_list=[]
    temp_string=''

    for i in text:
        if i.strip()!='':
            temp_list.append(re.findall('.+\t',i.strip())[0][:-1])
            temp_string=temp_string+re.findall('.+\t',i.strip())[0][:-1]+'omega_ts'
        else:
            tokenized.append(temp_list)
            tokenized_strings.append(temp_string)
            temp_list=[]
            temp_string=''

with open(os.getcwd()[:-8]+'multilexnorm/data/es/test.norm') as text:
    temp_list=[]
    temp_string=''

    for i in text:
        if i.strip()!='':
            temp_list.append(re.findall('.+\t',i.strip())[0][:-1])
            temp_string=temp_string+re.findall('.+\t',i.strip())[0][:-1]+'omega_ts'
        else:
            tokenized.append(temp_list)
            tokenized_strings.append(temp_string)
            temp_list=[]
            temp_string=''


In [4]:
len(tokenized)

1099

In [5]:
def lower_case(text):
    """
    This makes all upper case letters lower
    """
    new_text=''
    for i in text:
        if i.isupper():
            new_text=new_text+i.lower()
        else:
            new_text=new_text+i
    return new_text

In [6]:
def kill_symbols(text):
    """
    Function to get rid of annoying symbols in a token
    """
    symbols=['.', '+', '*', '?', '^', '$', '(', ')', '[', ']', '{', '}', '|', '\\',';',':','',"'",'-','!']
    new_token=''
    for letter in text:
        if letter in symbols:
            new_token=new_token+'\\'+letter
        else:
            new_token=new_token+letter
    return new_token

In [7]:
def cut_thing(j):
    """
    This function will cut off the raw string based on the last token and the count of that token from a tokenized list
    """
    symbols=['.', '+', '*', '?', '^', '$', '(', ')', '[', ']', '{', '}', '|', '\\',';',':','',"'",'-','!']
    last_token=j[0][-1]
    count_of_last=-1
    og_sent=j[1]
    new_sent=lower_case(j[1])
    if set(last_token) & set(symbols):
        last_token=kill_symbols(last_token)
    for unique in j[0]:
            if re.search(last_token,unique):
                count_of_last+=len([x for x in re.finditer(last_token,unique)])

    positions=[x for x in re.finditer(last_token,new_sent)]
    
    cut_off=og_sent[:positions[count_of_last].span()[1]+1][:-1]
    return cut_off

In [8]:
def find_phrase(t_phrase, phrase):
    """
    This function returns a score for how many consecutive tokens were found
    """
    symbols=['.', '+', '*', '?', '^', '$', '(', ')', '[', ']', '{', '}', '|', '\\',';',':','',"'",'-','!']
    count=0
    user_count=0
    cont=0
    cont_flag=1
    same_flag=0
    t_length=sum([len(x)+1 for x in t_phrase])
    used_spans=[]
    t0=kill_symbols(t_phrase[0])
    t1=kill_symbols(t_phrase[1])

    if re.search(t0,t1):
        used_spans.append(re.search(t0, phrase).span()[0])
        if re.search(t1,phrase):
            if re.search(t1,phrase).span()[0] not in used_spans:
                count+=1
            
    for token in t_phrase:
        if set(token) & set(symbols):
            token=kill_symbols(token)
        if re.search(token, phrase):
            if re.search(token, phrase).span()[0]>t_length:
                cont_flag=0
                cont=0
            count=count+1+cont
            cont+=1*cont_flag
            
    return count


In [9]:
symbols=['.', '+', '*', '?', '^', '$', '(', ')', '[', ']', '{', '}', '|', '\\',';',':','',"'",'-','!']
counter=-1
allcounts=[]
matches=[]
for t_phrase,t_string in zip(tokenized,tokenized_strings):
    lengths=[]
    counter+=1
    tokens=len(t_phrase)
    found=0
    first_token=t_phrase[0]
    if set(first_token) & set(symbols):
        first_token=kill_symbols(first_token)
    #get all places in the text where the first token is present and save the next 560 characters from there as a possible match
    possible_matches=[x for x in re.finditer(first_token,alltext_lower)]
    first_places=[x.span()[0] for x in possible_matches]
    possible_phrases=[alltext[x:x+560] for x in first_places]
    possible_phrases_lower=[alltext_lower[x:x+560] for x in first_places]
    
    #assign all possible matches scores
    counts=[]
    for phrase in possible_phrases_lower:
        number=find_phrase(t_phrase, phrase)
        counts.append(number)
        if number>=max(counts):
            try:
                #this is just to make sure that in case of a weird edge case we know the length of the contenders
                lengths.append([len(cut_thing([t_phrase,phrase])),phrase,number])
            except:
                pass
    lengths=[x for x in lengths if x[2]==max(counts)]
    if counts!=[]:
        if counts.count(max(counts))!=1:#dealing with edge case where we need to prioritize shorter sentence
            temp_len=[x[0] for x in lengths]
            min_index=temp_len.index(min(temp_len))
            matches.append([t_phrase,lengths[min_index][1],t_string])

        else:         
            max_index=counts.index(max(counts))
            matches.append([t_phrase,possible_phrases[max_index],t_string])
            allcounts.append(max(counts))

    else:
        matches.append([t_phrase,'nothing found',t_string])

In [10]:
counter=0
#check that everything is found
for i in matches:
    if i[1]=='nothing found':
        print(i)
        counter+=1
print(counter)

0


In [11]:
cut_matches=[]
#cut off based on the last token and save to disk
for j in matches:
    cut_matches.append([j[2],cut_thing(j)])

In [12]:
col1=[]
col2=[]
for i in cut_matches:
    col1.append(i[1])
    col2.append(i[0])
matched_df=pd.DataFrame([col1,col2])
matched_df=matched_df.T
matched_df.columns=['Original','Tokenized']

In [13]:
matched_df.to_csv(os.getcwd()[:-8]+'matched/matched_es.csv',index=False)

In [14]:
#sanity check
matched_df[150:200]

,Original,Tokenized
150,PARA MI YA NO ERES NADA SOLO UNA SUCIA RAMERA.,paraomega_tsmiomega_tsyaomega_tsnoomega_tseres...
151,@adricastro6 nbeeeaaa y eso? Que le pasa a tu ...,@adricastro6omega_tsnbeeeaaaomega_tsyomega_tse...
152,Con ganas de que venga a verme @JuananMarley m...,conomega_tsganasomega_tsdeomega_tsqueomega_tsv...
153,Lo malo no es trompezar con la piedra si no qu...,loomega_tsmaloomega_tsnoomega_tsesomega_tstrom...
154,Tengo ganas de que llegues ah casa yo mas....,tengoomega_tsganasomega_tsdeomega_tsqueomega_t...
155,Loo Exoo Maazooo dee Menooss Puuff :(,looomega_tsexooomega_tsmaazoooomega_tsdeeomega...
156,OYE OYE. K PASA AKI EH. :'( @Lauuuura99 @TriiHDD,oyeomega_tsoye.omega_tskomega_tspasaomega_tsak...
157,"Ya sabes que no resisto mucho una mirada, much...",yaomega_tssabesomega_tsqueomega_tsnoomega_tsre...
158,""" La sonrisa mas bella llega después de la lag...","""omega_tslaomega_tssonrisaomega_tsmasomega_tsb..."
159,@mariagodino94 @kimbherlynmo :O al vais a band...,@mariagodino94omega_ts@kimbherlynmoomega_ts:oo...


In [15]:
count=0
for i in matched_df.iterrows():
    for j in i[1]['Tokenized'].split('omega_ts')[:-1]:
        count+=1
print(count)

13824
